In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('../../../Dataset/WIDER_FACE/img_list_train.csv')

In [1]:
import pickle

In [2]:
with open('label_train.pickle',"rb") as fr:
    test_dict = pickle.load(fr)

In [3]:
test_dict

{'32--Worker_Laborer/32_Worker_Laborer_Worker_Laborer_32_80.jpg': {'size': (1024,
   677),
  'bbox': array([[394,  67,  49,  50],
         [669, 161,  47,  55]])},
 '32--Worker_Laborer/32_Worker_Laborer_Worker_Laborer_32_696.jpg': {'size': (1024,
   654),
  'bbox': array([[818, 100, 112, 136],
         [128, 132, 102, 142]])},
 '32--Worker_Laborer/32_Worker_Laborer_Worker_Laborer_32_445.jpg': {'size': (1024,
   652),
  'bbox': array([[536,  54,  68,  88],
         [828,  84,  72,  98],
         [ 88, 116,  54,  68]])},
 '32--Worker_Laborer/32_Worker_Laborer_Worker_Laborer_32_948.jpg': {'size': (1024,
   691),
  'bbox': array([[671, 234,  38,  54]])},
 '32--Worker_Laborer/32_Worker_Laborer_Worker_Laborer_32_360.jpg': {'size': (1024,
   682),
  'bbox': array([[346,  92, 254, 366]])},
 '32--Worker_Laborer/32_Worker_Laborer_Worker_Laborer_32_628.jpg': {'size': (1024,
   1579),
  'bbox': array([[336, 213,  99, 139]])},
 '32--Worker_Laborer/32_Worker_Laborer_Worker_Laborer_32_50.jpg': {'size

In [18]:
df.loc[df['image_path'] == '32--Worker_Laborer/32_Worker_Laborer_Worker_Laborer_32_80.jpg', ['height', 'width']]

,height,width
0,677,1024
